In [22]:

# Sample Python code to analyze
sample_code = r"""
def e2e_eval(gt_dir, res_dir, ignore_blank=False):
    print('start testing...')
    iou_thresh = 0.5
    val_names = os.listdir(gt_dir)
    num_gt_chars = 0
    gt_count = 0
    dt_count = 0
    hit = 0
    ed_sum = 0
    for i, val_name in enumerate(val_names):
        with open(os.path.join(gt_dir, val_name), encoding='utf-8') as f:
            gt_lines = [o.strip() for o in f.readlines()]
        gts = []
        ignore_masks = []
        for line in gt_lines:
            parts = line.strip().split('\t')
            # ignore illegal data
            if len(parts) < 9:
                continue
            assert (len(parts) < 11)
            if len(parts) == 9:
                gts.append(parts[:8] + [''])
            else:
                gts.append(parts[:8] + [parts[-1]])
            ignore_masks.append(parts[8])
        val_path = os.path.join(res_dir, val_name)
        if not os.path.exists(val_path):
            dt_lines = []
        else:
            with open(val_path, encoding='utf-8') as f:
                dt_lines = [o.strip() for o in f.readlines()]
        dts = []
        for line in dt_lines:
            # print(line)
            parts = line.strip().split("\t")
            assert (len(parts) < 10), "line error: {}".format(line)
            if len(parts) == 8:
                dts.append(parts + [''])
            else:
                dts.append(parts)
        dt_match = [False] * len(dts)
        gt_match = [False] * len(gts)
        all_ious = defaultdict(tuple)
        for index_gt, gt in enumerate(gts):
            gt_coors = [float(gt_coor) for gt_coor in gt[0:8]]
            gt_poly = polygon_from_str(gt_coors)
            for index_dt, dt in enumerate(dts):
                dt_coors = [float(dt_coor) for dt_coor in dt[0:8]]
                dt_poly = polygon_from_str(dt_coors)
                iou = polygon_iou(dt_poly, gt_poly)
                if iou >= iou_thresh:
                    all_ious[(index_gt, index_dt)] = iou
        sorted_ious = sorted(
            all_ious.items(), key=operator.itemgetter(1), reverse=True)
        sorted_gt_dt_pairs = [item[0] for item in sorted_ious]

        # matched gt and dt
        for gt_dt_pair in sorted_gt_dt_pairs:
            index_gt, index_dt = gt_dt_pair
            if gt_match[index_gt] == False and dt_match[index_dt] == False:
                gt_match[index_gt] = True
                dt_match[index_dt] = True
                if ignore_blank:
                    gt_str = strQ2B(gts[index_gt][8]).replace(" ", "")
                    dt_str = strQ2B(dts[index_dt][8]).replace(" ", "")
                else:
                    gt_str = strQ2B(gts[index_gt][8])
                    dt_str = strQ2B(dts[index_dt][8])
                if ignore_masks[index_gt] == '0':
                    ed_sum += ed(gt_str, dt_str)
                    num_gt_chars += len(gt_str)
                    if gt_str == dt_str:
                        hit += 1
                    gt_count += 1
                    dt_count += 1

        # unmatched dt
        for tindex, dt_match_flag in enumerate(dt_match):
            if dt_match_flag == False:
                dt_str = dts[tindex][8]
                gt_str = ''
                ed_sum += ed(dt_str, gt_str)
                dt_count += 1

        # unmatched gt
        for tindex, gt_match_flag in enumerate(gt_match):
            if gt_match_flag == False and ignore_masks[tindex] == '0':
                dt_str = ''
                gt_str = gts[tindex][8]
                ed_sum += ed(gt_str, dt_str)
                num_gt_chars += len(gt_str)
                gt_count += 1

    eps = 1e-9
    print('hit, dt_count, gt_count', hit, dt_count, gt_count)
    precision = hit / (dt_count + eps)
    recall = hit / (gt_count + eps)
    fmeasure = 2.0 * precision * recall / (precision + recall + eps)
    avg_edit_dist_img = ed_sum / len(val_names)
    avg_edit_dist_field = ed_sum / (gt_count + eps)
    character_acc = 1 - ed_sum / (num_gt_chars + eps)
    print('character_acc: %.2f' % (character_acc * 100) + "%")
    print('avg_edit_dist_field: %.2f' % (avg_edit_dist_field))
    print('avg_edit_dist_img: %.2f' % (avg_edit_dist_img))
    print('precision: %.2f' % (precision * 100) + "%")
    print('recall: %.2f' % (recall * 100) + "%")
    print('fmeasure: %.2f' % (fmeasure * 100) + "%")
"""

In [9]:
import ast

class ScopeHeatmapGenerator(ast.NodeVisitor):
    def __init__(self, code):
        self.code = code
        self.scopes = []  # A list of sets, where each set represents variables in a scope
        self.active_vars_per_line = [0] * (len(code.split('\n')) + 1)  # Initialize the count of active variables per line

    def enter_scope(self):
        if not self.scopes:
            self.scopes.append(set())  # Create the first global scope
        else:
            # New scope inherits all variables from the previous scope
            self.scopes.append(self.scopes[-1].copy())

    def exit_scope(self):
        self.scopes.pop()  # Remove the top scope when exiting

    def visit_FunctionDef(self, node):
        self.enter_scope()
        for arg in node.args.args:
            self.scopes[-1].add(arg.arg)
        self.generic_visit(node)
        self.exit_scope()

    def visit_AsyncFunctionDef(self, node):
        self.enter_scope()
        for arg in node.args.args:
            self.scopes[-1].add(arg.arg)
        self.generic_visit(node)
        self.exit_scope()

    def visit_For(self, node):
        self.enter_scope()
        if isinstance(node.target, ast.Name):
            self.scopes[-1].add(node.target.id)
        self.generic_visit(node)
        self.exit_scope()

    def visit_Assign(self, node):
        for target in node.targets:
            if isinstance(target, ast.Name):
                self.scopes[-1].add(target.id)
        self.generic_visit(node)

    def generic_visit(self, node):
        """Called if no explicit visitor function exists for a node."""
        if hasattr(node, 'lineno'):
            # Count the number of variables in scope at this line
            self.active_vars_per_line[node.lineno] = len(self.scopes[-1])
        super().generic_visit(node)

    def analyze(self):
        """Analyze the given code to generate the scope heatmap."""
        # Parse the code into an AST
        tree = ast.parse(self.code)
        # Start with the global scope
        self.enter_scope()
        # Visit all nodes
        self.visit(tree)
        # Exit the global scope
        self.exit_scope()
        
        # Normalize the heatmap data to remove any lines beyond the code length
        code_length = len(self.code.split('\n'))
        self.active_vars_per_line = self.active_vars_per_line[:code_length + 1]

        return self.active_vars_per_line

# Initialize the heatmap generator with the sample code
heatmap_generator = ScopeHeatmapGenerator(sample_code)

# Generate the heatmap
heatmap_data = heatmap_generator.analyze()
heatmap_data

[0,
 0,
 3,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 11,
 12,
 13,
 14,
 15,
 0,
 15,
 15,
 15,
 15,
 15,
 0,
 15,
 15,
 14,
 14,
 15,
 0,
 15,
 15,
 16,
 17,
 0,
 18,
 18,
 18,
 18,
 0,
 18,
 17,
 18,
 19,
 19,
 20,
 21,
 21,
 22,
 23,
 24,
 24,
 24,
 20,
 20,
 21,
 0,
 0,
 22,
 22,
 22,
 22,
 22,
 22,
 23,
 24,
 0,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 0,
 0,
 21,
 21,
 22,
 23,
 23,
 23,
 0,
 0,
 21,
 21,
 22,
 23,
 23,
 23,
 23,
 0,
 11,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 0]

In [11]:
def visualize_heatmap(code, heatmap_data):
    lines = code.split('\n')
    max_heat = max(heatmap_data)
    # Normalize heat levels to a scale of 0-10
    normalized_heatmap = [int((heat / max_heat) * 10) if max_heat > 0 else 0 for heat in heatmap_data]

    # Generate visualization for each line
    for i, line in enumerate(lines, start=1):
        heat_bar = '█' * normalized_heatmap[i - 1]
        print(f"{str(i).rjust(2)}: {line[:80].ljust(80)} | {heat_bar}")

# Visualize the heatmap alongside the code
visualize_heatmap(sample_code, heatmap_data)

 1:                                                                                  | 
 2: def e2e_eval(gt_dir, res_dir, ignore_blank=False):                               | 
 3:     print('start testing...')                                                    | █
 4:     iou_thresh = 0.5                                                             | █
 5:     val_names = os.listdir(gt_dir)                                               | █
 6:     num_gt_chars = 0                                                             | ██
 7:     gt_count = 0                                                                 | ██
 8:     dt_count = 0                                                                 | ██
 9:     hit = 0                                                                      | ███
10:     ed_sum = 0                                                                   | ███
11:     for i, val_name in enumerate(val_names):                                     | ████
12:         w

In [23]:
import ast

# Parse the Python code into an AST
parsed_code = ast.parse(sample_code)

# Helper function to traverse the AST and find the spans
def find_usage_spans(node, usage_dict=None, parent_function=None):
    if usage_dict is None:
        usage_dict = {}
    
    for child in ast.iter_child_nodes(node):
        if isinstance(child, ast.Name):
            name = child.id
            if name not in usage_dict:
                usage_dict[name] = {'first_used_line': child.lineno, 'last_used_line': child.lineno, 'parent_function': parent_function}
            else:
                usage_dict[name]['last_used_line'] = child.lineno
        
        elif isinstance(child, ast.FunctionDef):
            parent_function = child.name
        
        find_usage_spans(child, usage_dict, parent_function)
    
    return usage_dict

# Finding the usage spans
usage_spans = find_usage_spans(parsed_code)

usage_spans

{'print': {'first_used_line': 3,
  'last_used_line': 108,
  'parent_function': 'e2e_eval'},
 'iou_thresh': {'first_used_line': 4,
  'last_used_line': 52,
  'parent_function': 'e2e_eval'},
 'val_names': {'first_used_line': 5,
  'last_used_line': 100,
  'parent_function': 'e2e_eval'},
 'os': {'first_used_line': 5,
  'last_used_line': 28,
  'parent_function': 'e2e_eval'},
 'gt_dir': {'first_used_line': 5,
  'last_used_line': 12,
  'parent_function': 'e2e_eval'},
 'num_gt_chars': {'first_used_line': 6,
  'last_used_line': 102,
  'parent_function': 'e2e_eval'},
 'gt_count': {'first_used_line': 7,
  'last_used_line': 101,
  'parent_function': 'e2e_eval'},
 'dt_count': {'first_used_line': 8,
  'last_used_line': 97,
  'parent_function': 'e2e_eval'},
 'hit': {'first_used_line': 9,
  'last_used_line': 98,
  'parent_function': 'e2e_eval'},
 'ed_sum': {'first_used_line': 10,
  'last_used_line': 102,
  'parent_function': 'e2e_eval'},
 'i': {'first_used_line': 11,
  'last_used_line': 11,
  'parent_f

In [40]:
# Function to determine the number of active variables at each line
def active_variables_per_line(usage_spans):
    # Initialize a dictionary to count active variables per line
    active_vars = {}

    # Iterate through each variable and its span
    for var, spans in usage_spans.items():
        # If the variable is within a function, we'll only count it within that function's scope
        # for line in (spans['first_used_line'], spans['last_used_line'] + 1):
        for line in range(spans['first_used_line'], spans['last_used_line'] + 1):
        # for line in (spans['last_used_line'] + 1,):
            key = line
            if key in active_vars:
                active_vars[key].add(var)
            else:
                active_vars[key] = {var}
    
    # Convert sets to counts
    active_vars_counts = {key: len(vars) for key, vars in active_vars.items()}
    
    return active_vars_counts

# Determine the number of active variables for each line number
active_vars_counts = active_variables_per_line(usage_spans)

# Since the function above creates a tuple (function_name, line) as key when in a function scope,
# we'll create a more readable format by concatenating the function name and line number.
formatted_active_vars_counts = {
    f"line {key}": count
    for key, count in active_vars_counts.items()
}

formatted_active_vars_counts

{'line 3': 1,
 'line 4': 2,
 'line 5': 5,
 'line 6': 6,
 'line 7': 7,
 'line 8': 8,
 'line 9': 9,
 'line 10': 10,
 'line 11': 13,
 'line 12': 14,
 'line 13': 15,
 'line 14': 16,
 'line 15': 17,
 'line 16': 18,
 'line 17': 18,
 'line 18': 18,
 'line 19': 19,
 'line 20': 19,
 'line 21': 19,
 'line 22': 19,
 'line 23': 19,
 'line 24': 19,
 'line 25': 19,
 'line 26': 19,
 'line 27': 21,
 'line 28': 19,
 'line 29': 19,
 'line 30': 19,
 'line 31': 19,
 'line 32': 17,
 'line 33': 16,
 'line 34': 16,
 'line 35': 15,
 'line 36': 15,
 'line 37': 15,
 'line 38': 14,
 'line 39': 14,
 'line 40': 14,
 'line 41': 14,
 'line 42': 14,
 'line 43': 15,
 'line 44': 18,
 'line 45': 18,
 'line 46': 21,
 'line 47': 21,
 'line 48': 22,
 'line 49': 24,
 'line 50': 22,
 'line 51': 22,
 'line 52': 19,
 'line 53': 18,
 'line 54': 19,
 'line 55': 19,
 'line 56': 19,
 'line 57': 17,
 'line 58': 17,
 'line 59': 18,
 'line 60': 17,
 'line 61': 16,
 'line 62': 16,
 'line 63': 16,
 'line 64': 17,
 'line 65': 18,
 'line

In [41]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from pygments.styles import get_style_by_name
import IPython

# Function to generate HTML representation of the code with heatmap colors
def generate_heatmap_html(code, active_vars_counts):
    # Define the style for syntax highlighting
    style = get_style_by_name('friendly')
    formatter = HtmlFormatter(style=style)
    # Get the tokenized code in HTML
    highlighted_code = highlight(code, PythonLexer(), formatter)
    
    # Start with the style information
    html_code = formatter.get_style_defs()
    
    # Add the pre-wrapped code
    html_code += '<pre style="line-height: 125%">'

    max_count = max(active_vars_counts.values())
    
    # Go through each line and wrap it with a span that has a background color corresponding to the number of active vars
    for i, line in enumerate(code.splitlines(), 1):
        count = active_vars_counts.get(i, 0)
        color_intensity = 255 - min(int(200 * count / max_count), 255)  # Scale the color intensity
        bg_color = f'rgb(255,{color_intensity},{color_intensity})'  # More vars -> more intense red
        html_code += f'<span style="background-color: {bg_color}">{line}\n</span>'
    
    html_code += '</pre>'
    return html_code

# Generate HTML with heatmap
html_with_heatmap = generate_heatmap_html(sample_code, active_vars_counts)

# Display the HTML in the IPython output cell
IPython.display.HTML(html_with_heatmap)
# print(html_with_heatmap)
